In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [30]:

def read_data(filename):
    data = pd.read_csv(filename)
    data["timestamp"] = pd.to_datetime(data["timestamp"]).dt.date
    data = data.set_index("timestamp")
    return data


In [31]:
data = read_data("../data/TSLA_daily_EWMA.csv")

In [32]:
def xy_matrix(xcolumn1, xcolumn2, ycolumn, data):
    X = data[[xcolumn1, xcolumn2]]
    X = X.drop(X.index[1258])
    y = data[[ycolumn]].shift(1)
    y = y.dropna()
    return X, y

In [33]:
xy_matrix('8EWMA', '20EWMA', 'Signal', data)

(                 8EWMA      20EWMA
 timestamp                         
 2017-06-02  339.850000  339.850000
 2017-06-05  344.051875  343.771750
 2017-06-06  347.744352  347.105229
 2017-06-07  351.917053  350.726710
 2017-06-08  357.534316  355.388755
 ...                ...         ...
 2022-05-24  698.958569  777.746593
 2022-05-25  690.034443  766.418346
 2022-05-26  693.966789  760.828980
 2022-05-27  708.558614  760.714791
 2022-05-31  719.603366  760.481001
 
 [1258 rows x 2 columns],
             Signal
 timestamp         
 2017-06-05     0.0
 2017-06-06     1.0
 2017-06-07     1.0
 2017-06-08     1.0
 2017-06-09     1.0
 ...            ...
 2022-05-25     0.0
 2022-05-26     0.0
 2022-05-27     0.0
 2022-05-31     0.0
 2022-06-01     0.0
 
 [1258 rows x 1 columns])

In [34]:
X,y = xy_matrix('8EWMA', '20EWMA', 'Signal', data)

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test) 

In [36]:
def ann_function(X_train_scaled,X_test_scaled,y_train,y_test):
     ann = tf.keras.models.Sequential()
     ann.add(tf.keras.layers.Dense(units=100,activation="sigmoid"))
     ann.add(tf.keras.layers.Dense(units=50,activation="sigmoid"))
     ann.add(tf.keras.layers.Dense(units=40,activation="tanh"))
     ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
     ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
     ann.fit(X_train_scaled,y_train,batch_size=32,epochs = 500)
     ann_loss, ann_accuracy = ann.evaluate(X_test_scaled, y_test, verbose=2)
     predictions = ann.predict(X_test_scaled)
     return predictions, ann_loss, ann_accuracy


In [37]:
ann_function(X_train_scaled,X_test_scaled,y_train,y_test)

Epoch 1/500
32/32 [==============================] - 1s 2ms/step - loss: 0.6969 - accuracy: 0.5119
Epoch 2/500
32/32 [==============================] - 0s 2ms/step - loss: 0.6875 - accuracy: 0.5467
Epoch 3/500
32/32 [==============================] - 0s 2ms/step - loss: 0.6825 - accuracy: 0.5447
Epoch 4/500
32/32 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.5477
Epoch 5/500
32/32 [==============================] - 0s 2ms/step - loss: 0.6707 - accuracy: 0.5636
Epoch 6/500
32/32 [==============================] - 0s 2ms/step - loss: 0.6748 - accuracy: 0.5915
Epoch 7/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6774 - accuracy: 0.5746
Epoch 8/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6687 - accuracy: 0.5785
Epoch 9/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6672 - accuracy: 0.5895
Epoch 10/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6650 - accuracy: 0.5815
Epoch 11/

(array([[1.3376520e-05],
        [1.0000000e+00],
        [1.0000000e+00],
        [1.7901891e-01],
        [5.9896385e-12],
        [3.8158596e-03],
        [1.0000000e+00],
        [2.9762518e-06],
        [9.9999875e-01],
        [2.1476406e-01],
        [1.0000000e+00],
        [1.0963960e-06],
        [1.0000000e+00],
        [3.2585838e-09],
        [1.0000000e+00],
        [9.9999809e-01],
        [1.0000000e+00],
        [9.9999875e-01],
        [1.0000000e+00],
        [9.9998617e-01],
        [1.0000000e+00],
        [1.0000000e+00],
        [9.9988067e-01],
        [5.2422082e-01],
        [9.7421139e-01],
        [9.7059608e-03],
        [3.5807490e-04],
        [7.7224895e-06],
        [9.9998999e-01],
        [1.6894758e-02],
        [1.0000000e+00],
        [1.7696095e-01],
        [2.4460645e-10],
        [9.3693686e-01],
        [9.9781966e-01],
        [2.1060711e-36],
        [1.0000000e+00],
        [9.9924266e-01],
        [4.9005905e-37],
        [8.7812577e-06],


In [40]:
predictions_df = pd.DataFrame(ann_function(X_train_scaled,X_test_scaled,y_train,y_test), dtype=object)

Epoch 1/500
32/32 [==============================] - 1s 2ms/step - loss: 0.6998 - accuracy: 0.5000
Epoch 2/500
32/32 [==============================] - 0s 4ms/step - loss: 0.6877 - accuracy: 0.5408
Epoch 3/500
32/32 [==============================] - 0s 4ms/step - loss: 0.6861 - accuracy: 0.5567
Epoch 4/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6764 - accuracy: 0.5676
Epoch 5/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6778 - accuracy: 0.5656
Epoch 6/500
32/32 [==============================] - 0s 4ms/step - loss: 0.6710 - accuracy: 0.5905
Epoch 7/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6687 - accuracy: 0.5815
Epoch 8/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6713 - accuracy: 0.5875
Epoch 9/500
32/32 [==============================] - 0s 2ms/step - loss: 0.6838 - accuracy: 0.5427
Epoch 10/500
32/32 [==============================] - 0s 3ms/step - loss: 0.6738 - accuracy: 0.5974
Epoch 11/

c:\Users\Ldkel\anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [41]:
predictions_df

,0
0,"[[0.00013157725], [1.0], [1.0], [0.66218436], ..."
1,0.212533
2,0.892857
